From [naturalscenesdataset.org](https://naturalscenesdataset.org):
```text
The Natural Scenes Dataset (NSD) is a large-scale fMRI dataset conducted at ultra-high-field (7T) strength at the Center of Magnetic Resonance Research (CMRR) at the University of Minnesota. The dataset consists of whole-brain, high-resolution (1.8-mm isotropic, 1.6-s sampling rate) fMRI measurements of 8 healthy adult subjects while they viewed thousands of color natural scenes over the course of 30–40 scan sessions. While viewing these images, subjects were engaged in a continuous recognition task in which they reported whether they had seen each given image at any point in the experiment. These data constitute a massive benchmark dataset for computational models of visual representation and cognition, and can support a wide range of scientific inquiry.
```


In [2]:
from cloudpathlib import S3Path, S3Client
from pathlib import Path

# Set up our cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

# Create the root S3Path for the NSD:
nsd_base_path = S3Path(
    's3://natural-scenes-dataset/',
    client=S3Client(
        no_sign_request=True,
        local_cache_dir=cache_path))

In [3]:
import os
import os.path as op
import glob
import nibabel as nib
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from nilearn import plotting

import urllib.request, zipfile
try:
    from pycocotools.coco import COCO
except ImportError as e:
    !pip install pycocotools
    from pycocotools.coco import COCO


#from nsd_access import NSDAccess

%matplotlib inline

In [4]:
from utils import ls, crawl

nsd_pppath = nsd_base_path / 'nsddata_betas' / 'ppdata'

ls(nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR')

[S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session01.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session02.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session03.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session04.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session05.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session06.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session07.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh

### Loading data in volume space (1.8mm isotropic resolution)

In [13]:
vimg_filename = nsd_pppath / 'subj01' / 'func1pt8mm' / 'betas_fithrf_GLMdenoise_RR' / 'betas_session01.nii.gz'
vimg = nib.load(vimg_filename.fspath)
vimg = vimg.dataobj

# reshape to vector of nvoxels x ntimepoints
vimg = vimg.reshape([np.prod(vimg.shape[0:-1]),vimg.shape[-1]])

vimg.shape

### Loading data in surface space (fsaverage)

In [14]:
simg_filename = nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR' / 'lh.betas_session01.mgh'
simg = nib.load(simg_filename.fspath)
simg = simg.dataobj

# reshape to vector of nvoxels x ntimepoints
simg = simg.reshape([np.prod(simg.shape[0:-1]),simg.shape[-1]])

simg.shape